In [1]:
# Считайте файл согласно вашему варианту и подсчитайте среднее арифметическое чисел по каждой строке. 
# В результирующем файле выведите полученные данные:


# average1
# average2
# average3
# -----------
# averageN


In [7]:
filename = "text_2_var_17"

with open('text_2_var_17') as f:
    lines = f.readlines()
    
sum_lines = []

for i in lines:
        nums = i.split(".")
        avage = 0
        sum_line = 0
        for j in nums:
            sum_line += int(j)
            avage = sum_line // len(nums)
        sum_lines.append(avage)
print(sum_lines)

with open(filename+"result", 'w') as f:
    for k in sum_lines:
        f.write(str(k) + '\n')

[81691899, -175346216, -90170327, -42993726, 255348223, 46794686, -17965310, 51303427, -236141619, -31799427, -39739111, -28601432, -107570191, -18591511, 431831658, 279454598, 131896342, 196724133, 190333043, 69986173, 120908053, -80564389, -57789889, 283435167, -77846583, -272244788, -124268281, 83090610, 165040934, -10183432, 176750376, -95879028, 427782613, -212825082, 4744410, -55489705, 82347719, -95728039, -204713299, -38770152, -66686256, 514403571, 91597206, 139644631, 53186947, -139744469, 66724550, -267273592, 267793136, -257021480, -50579808, -137653709, -284931229, 144782638, 96189331, -163835910]
